In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Sample Spam Dataset (Replace with your own dataset if available)
data = {
    "text": [
        "Win a free iPhone now!", "Congrats! You won a lottery!", "Urgent: Claim your prize today!",
        "Hello, how are you?", "Let's have a meeting tomorrow.", "Are we still on for lunch?",
        "Limited time offer! Click here!", "Cheap loans available!", "Buy 1 get 1 free today!",
        "Good morning, have a great day!"
    ],
    "label": ["spam", "spam", "spam", "ham", "ham", "ham", "spam", "spam", "spam", "ham"]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Encode Labels (Spam = 1, Ham = 0)
encoder = LabelEncoder()
df["label"] = encoder.fit_transform(df["label"])

# Splitting into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42)

# Tokenizing Text Data
max_words = 1000  # Vocabulary size
max_len = 20  # Maximum sequence length

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len, padding="post")
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len, padding="post")

# Building the ANN Model
model = Sequential([
    Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")  # Output layer for binary classification
])

# Compile Model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the Model
model.fit(X_train_seq, y_train, epochs=10, validation_data=(X_test_seq, y_test), verbose=2)

# Evaluate the Model
loss, accuracy = model.evaluate(X_test_seq, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# Making Predictions
sample_texts = ["Congratulations, you won a free gift!", "Let's meet for coffee."]
sample_seq = pad_sequences(tokenizer.texts_to_sequences(sample_texts), maxlen=max_len, padding="post")

predictions = model.predict(sample_seq)
pred_labels = ["Spam" if p > 0.5 else "Ham" for p in predictions]

# Display Predictions
for text, label in zip(sample_texts, pred_labels):
    print(f"Message: {text} → Prediction: {label}")

